### Generalized Odd-Even Sort with Semaphores (Java) [4 points]

In odd-even sort, pairs of consecutive elements are swapped in phases: First, the pairs at indices (0, 1), (2, 3), (4, 5), ... are swapped if necessary, then those at indices (1, 2), (3, 4), (5, 6), ..., then this is repeated until all elements are sorted:

| 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | index   |
| - | - | - | - | - | - | - | - | - | - |:------- |
| 8 | 1 | 7 | 0 | 4 | 5 | 3 | 9 | 6 | 2 | initial |
| 1 | 8 | 0 | 7 | 4 | 5 | 3 | 9 | 2 | 6 | swapping at even-odd indices |
| 1 | 0 | 8 | 4 | 7 | 3 | 5 | 2 | 9 | 6 | swapping at odd-even indices |
| 0 | 1 | 4 | 8 | 3 | 7 | 2 | 5 | 6 | 9 | swapping at even-odd indices |
| 0 | 1 | 4 | 3 | 8 | 2 | 7 | 5 | 6 | 9 | swapping at odd-even indices |
| 0 | 1 | 3 | 4 | 2 | 8 | 5 | 7 | 6 | 9 | swapping at even-odd indices |
| 0 | 1 | 3 | 2 | 4 | 5 | 8 | 6 | 7 | 9 | swapping at odd-even indices |
| 0 | 1 | 2 | 3 | 4 | 5 | 6 | 8 | 7 | 9 | swapping at even-odd indices |
| 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | swapping at odd-even indices |

Like bubble sort, the algorithm is based on swapping (sorting) consecutive elements. Here, all swaps in one phase can be done in parallel. If there are `N` elements, approximately `N / 2` processes can swap elements in parallel. If `N` is large, that would lead to a prohibitive number of processes. In the generalized odd-even sort, each process does not sort two consecutive elements but a subsequence. Suppose two processes are sorting the 10 elements above. In the first phase, one process sorts at indices 0 .. 3 and the other at 4 .. 7. In the second phase, one process sorts at indices 2 .. 5 and the other at 6 .. 9. Then, this is repeated until all elements are sorted:

| 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | index   |
| - | - | - | - | - | - | - | - | - | - |:------- |
| 8 | 1 | 7 | 0 | 4 | 5 | 3 | 9 | 6 | 2 | initial |
| 0 | 1 | 7 | 8 | 3 | 4 | 5 | 9 | 6 | 2 | sorting 0 .. 3 and 4 .. 7 |
| 0 | 1 | 3 | 4 | 7 | 8 | 2 | 5 | 6 | 9 | sorting 2 .. 5 and 6 .. 9 |
| 0 | 1 | 3 | 4 | 2 | 5 | 7 | 8 | 6 | 9 | sorting 0 .. 3 and 4 .. 7 |
| 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | sorting 2 .. 5 and 6 .. 9 |

Generalizing this to `N` elements and `P` processes, each process sorts approximately `N / P` elements sequentially. In the implementation below, quicksort is used for sequential sorting. Each round consists of two sorting phases. For simplicity, the array and the barriers are kept as global variables. To simplify the index calculations, `oddevensort` computes the list size to be sorted from the number of threads and the number of elements each thread sorts. A random list is then generated. You can uncomment some lines with test output to see the program work, but be aware that the output may be asynchronous.

In [7]:
%%writefile oddevensort.java

import java.util.Random;
import java.util.Arrays;
import java.util.concurrent.Semaphore;

public class oddevensort {
    public static int [] a;
    public static Semaphore[] barriers;

    public static int partition(int p, int r){
        int x = a[r];
        int i = p - 1;
        for (int j = p; j <= r - 1; j++) {
            if (a[j] <= x) {
                i++; 
                int t = a[i]; a[i] = a[j]; a[j] = t;
            }
        }
        int t = a[i + 1]; a[i + 1] = a[r]; a[r] = t;
        return i + 1;
    }

    public static void sequentialsort(int p, int r) {
        if (p < r) {
            int q = partition(p, r);
            sequentialsort(p, q - 1);
            sequentialsort(q + 1, r);
        }
    }

    public static void barriersync(int p){ // p is the current thread id
        try{
            for (int i = 0; i<barriers.length ; i++ ) if (i != p) barriers[i].release();
            for (int i = 0; i<barriers.length - 1; i++) barriers[p].acquire();
            
        } catch (Exception e) {}
    }

    public static Thread makesorter (int l0, int u0, int l1, int u1, int p, int r) {
        return new Thread () {
            public void run() {
                for (int i = 0; i < r; i++) {
                     System.out.printf("%d sorts %d to %d round  %d\n", p, l0, u0, i);
                    sequentialsort(l0, u0); barriersync(p);
                     System.out.printf("%d sorts %d to %d round  %d\n", p, l1, u1, i);
                    sequentialsort(l1, u1); barriersync(p);
                }
            }
        };
    }

    public static void main(String args[]) {
        int P = Integer.parseInt(args[0]); // number of sorting threads
        int M = Integer.parseInt(args[1]); // number of elements each thread sorts sequentially
        int N = P * M + M / 2;             // number of elements to be sorted
        System.out.println("size " + N);
        
        a = new int[N];
        Random random = new Random();
        for (int i = 0; i < N; i++) a[i] = random.nextInt(10000);
         System.out.println(Arrays.toString(a));

        barriers = new Semaphore[P];
        for (int p = 0; p < P; p++) barriers[p] = new Semaphore(0);

        Thread sorters[] = new Thread[P];
        for (int p = 0; p < P; p++) sorters[p] =
            makesorter(p * M, (p + 1) * M - 1, p * M + M / 2, (p + 1) * M + M / 2 - 1, p, P + 1);
        for (Thread s: sorters) s.start();
        try {for(Thread s: sorters) s.join();
        } catch (Exception e) {}
        
        for (int i = 1; i < N; i++) assert a[i - 1] <= a[i];
         System.out.println(Arrays.toString(a));
    }
}


Overwriting oddevensort.java


In [8]:
!javac oddevensort.java
!java -enableassertions oddevensort 2 10

size 25
[8198, 5726, 7579, 7210, 6357, 9753, 5571, 4668, 6551, 3193, 821, 1087, 199, 5521, 2675, 5568, 4104, 1802, 242, 4346, 3447, 9611, 9839, 2833, 7262]
0 sorts 0 to 9 round  0
1 sorts 10 to 19 round  0
1 sorts 15 to 24 round  0
0 sorts 5 to 14 round  0
0 sorts 0 to 9 round  1
1 sorts 10 to 19 round  1
1 sorts 15 to 24 round  1
0 sorts 5 to 14 round  1
0 sorts 0 to 9 round  2
1 sorts 10 to 19 round  2
1 sorts 15 to 24 round  2
0 sorts 5 to 14 round  2
[199, 242, 821, 1087, 1802, 2675, 2833, 3193, 3447, 4104, 4346, 4668, 5521, 5568, 5571, 5726, 6357, 6551, 7210, 7262, 7579, 8198, 9611, 9753, 9839]


In [6]:
!javac oddevensort.java
!java -enableassertions oddevensort 4 10

size 45
[7874, 2792, 1816, 4935, 7580, 8863, 8114, 5998, 7845, 8086, 8102, 1508, 1901, 1020, 9033, 222, 6429, 1401, 587, 3258, 5328, 6135, 9614, 4868, 1887, 5756, 6098, 4162, 9304, 8293, 8623, 906, 2042, 9359, 8254, 8179, 384, 7631, 6597, 7208, 1879, 9010, 4941, 649, 142]
[142, 222, 384, 587, 649, 906, 1020, 1401, 1508, 1816, 1879, 1887, 1901, 2042, 2792, 3258, 4162, 4868, 4935, 4941, 5328, 5756, 5998, 6098, 6135, 6429, 6597, 7208, 7580, 7631, 7845, 7874, 8086, 8102, 8114, 8179, 8254, 8293, 8623, 8863, 9010, 9033, 9304, 9359, 9614]
